In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

def createAvitoSearchPageUrl(category,searchQuery,page):
    domain = u'https://www.avito.ru/'
    searchQuery = searchQuery.replace(u' ',u'+')
    url = u'{0}{1}?q={2}&sgtd=2&p={3}'.format(domain,category,searchQuery,page)
    return url
def getSoupByUrl(url):
    proxies = {'http': 'http://94.77.88.2:3001',
          'https': 'https://94.77.88.2:3001'}
    return BeautifulSoup(requests.get(url, proxies=proxies).text,'lxml')
def checkIfPageIsNotContainsResults(soup):
    return u'ничего не найдено' in ''.join([tag.get_text() for tag in soup.find_all(attrs={'class':'nulus'})])
def getAdsFromAvitoSearchPage(soup):
    return soup.find(attrs={'class':'catalog-list'}).find_all(attrs={'class':'description'})
def getAdParams(soup):
    price = soup.find(attrs={'class':'about'}).get_text().replace('\n','').strip()
    shortAdData = soup.find(attrs={'class':'data'}).get_text().replace('\n','').strip()
    header = soup.find('h3').a['title']
    link = soup.find('h3').a['href']
    return {'header':header,'link':link,'shortAdData':shortAdData,'price':price,'link':link}
def getAdParamsFromAvitoSearchPage(ads):
    return [getAdParams(ad) for ad in ads]
def createAvitoAdPageUrl(link):
    domain = u'https://www.avito.ru'
    return '{0}{1}'.format(domain,link)
def getAdExtendedParamsBySoup(soup):
    todayDate = datetime.now().isoformat()
    header = soup.find('h1').get_text()
    placementDate = soup.find(attrs={'class':'item-subtitle'}).get_text().split('\n')[1].strip()
    price = soup.find(attrs={'itemprop':'price'}).get_text().strip()
    seller = soup.find(attrs={'id':'seller'}).get_text().strip().replace('\n','')
    city = soup.find(attrs={'itemprop':'availableAtOrFrom'}).find(attrs={'class':'description_content'}).get_text().replace('\n','').strip()
    descriptionText = soup.find(id='desc_text').get_text()
    adId = soup.find(id='item_id').get_text()
    return {'todayDate':todayDate,
            'header':header,
            'placementDate':placementDate,
            'price':price,
            'seller':seller,
            'city':city,
            'descriptionText':descriptionText,
            'adId':adId
           }
def getExtendedAdsParamsByListOfAds(ads):
    extendedAdsParams = []
    for ad in ads:
        adParams = getAdParams(ad)
        print adParams['link']
        try:
            soup = getSoupByUrl(createAvitoAdPageUrl(adParams['link']))
            extendedAdParams = getAdExtendedParamsBySoup(soup)
            extendedAdParams['link'] = adParams['link']
            extendedAdsParams.append(extendedAdParams)
        except:
            continue
    return extendedAdsParams

# category = u'moskovskaya_oblast/predlozheniya_uslug/transport_perevozki'
# query = u'грузовые перевозки'
# page = 1
# soup = getSoupByUrl(createAvitoSearchPageUrl(category,query,page))
# if checkIfPageIsNotContainsResults(soup):
#     ads = getAdsFromAvitoSearchPage(soup)
# allExtendedAdsParams = getExtendedAdsParamsByListOfAds(ads)
# allShortAdParams = []
# for ad in ads:
#     allShortAdParams.append(getAdParams(ad))
# df = pd.DataFrame(allShortAdParams)
# df.merge(pd.DataFrame(allExtendedAdsParams),on='link',how='outer').to_excel('D:\\avito_ads.xlsx')

In [2]:
searchQueries = []
with open(u'D:\\spisok_zaprosov.txt','r+') as f1:
    for row in f1:
            searchQueries.append(row.replace('\n','').lower().decode('utf-8'))

In [12]:
def getAvitoQueryDataFrame(query):
    category = u'moskva/predlozheniya_uslug/transport_perevozki'
    dataframe = pd.DataFrame()
    for page in xrange(1,10):
        print page
        soup = getSoupByUrl(createAvitoSearchPageUrl(category,query,page))
        if not checkIfPageIsNotContainsResults(soup):
            print 'ok'
            ads = getAdsFromAvitoSearchPage(soup)
            allExtendedAdsParams = getExtendedAdsParamsByListOfAds(ads)
            allShortAdParams = []
            for ad in ads:
                allShortAdParams.append(getAdParams(ad))
            df = pd.DataFrame(allShortAdParams)
            df = df.merge(pd.DataFrame(allExtendedAdsParams),on='link',how='outer')
            dataframe = pd.concat([dataframe,df])
    return dataframe

In [13]:
allData = pd.DataFrame()
for q in searchQueries:
    print q
    data = getAvitoQueryDataFrame(q)
    allData = pd.concat([allData,data])

откачка септиков
1
ok
/moskva/predlozheniya_uslug/uslugi_ilososa_748432283
/moskva/predlozheniya_uslug/uslugi_ilososa_kruglosutochno_739714165
/moskva/predlozheniya_uslug/otkachka_vygrebnyh_yam_ustranenie_zasorov_chistka_d_373368980
/moskva/predlozheniya_uslug/otkachka_ilososom_septikov_avtomoek_kanalizatsii_712663243
/moskva/predlozheniya_uslug/otkachka_septikov_tualetov_uslugi_735201395
/moskva/predlozheniya_uslug/uslugi_ilososov_otkachka_i_vyvoz_zhidkih_othodov_285816157
/moskva/predlozheniya_uslug/otkachka_zhidkostey_assenizatsiya_385293417
/moskva/predlozheniya_uslug/uslugi_vakuumnoy_mashiny_805956092
/moskva/predlozheniya_uslug/uslugi_ilososov_247_vsya_moskva_i_mo_629343756
/moskva/predlozheniya_uslug/uslugi_ilososa_-chistka_septika_avtomoyki_716420303
/moskva/predlozheniya_uslug/vyzov_assenizatora_ilososa_801691044
/moskva/predlozheniya_uslug/uslugi_ilososa_kamaz_458978260
/moskva/predlozheniya_uslug/ustranenie_zasorov_lyuboy_slozhnosti_gidrodinamika_727209892
/moskva/predlozhen

In [15]:
allData.to_excel('D:\\avito_ads2.xlsx')